In [88]:
from time import time

In [2]:
import tensorflow as tf
def as_keras_metric(method):
    import functools
    from keras import backend as K
    @functools.wraps(method)
    def wrapper(self, args, **kwargs):
        """ Wrapper for turning tensorflow metrics into keras metrics """
        value, update_op = method(self, args, **kwargs)
        K.get_session().run(tf.local_variables_initializer())
        with tf.control_dependencies([update_op]):
            value = tf.identity(value)
        return value
    return wrapper

precision = as_keras_metric(tf.metrics.precision)
recall = as_keras_metric(tf.metrics.recall)

Using TensorFlow backend.


In [3]:
from keras.datasets import imdb

vocabulary_size = 5000
(X_train, y_train), (X_test, y_test) = imdb.load_data(num_words = vocabulary_size)
print('Loaded dataset with {} training samples, {} test samples'.format(len(X_train), len(X_test)))

print('---review---')
print(X_train[6])
print('---label---')
print(y_train[6])

Loaded dataset with 25000 training samples, 25000 test samples
---review---
[1, 2, 365, 1234, 5, 1156, 354, 11, 14, 2, 2, 7, 1016, 2, 2, 356, 44, 4, 1349, 500, 746, 5, 200, 4, 4132, 11, 2, 2, 1117, 1831, 2, 5, 4831, 26, 6, 2, 4183, 17, 369, 37, 215, 1345, 143, 2, 5, 1838, 8, 1974, 15, 36, 119, 257, 85, 52, 486, 9, 6, 2, 2, 63, 271, 6, 196, 96, 949, 4121, 4, 2, 7, 4, 2212, 2436, 819, 63, 47, 77, 2, 180, 6, 227, 11, 94, 2494, 2, 13, 423, 4, 168, 7, 4, 22, 5, 89, 665, 71, 270, 56, 5, 13, 197, 12, 161, 2, 99, 76, 23, 2, 7, 419, 665, 40, 91, 85, 108, 7, 4, 2084, 5, 4773, 81, 55, 52, 1901]
---label---
1


In [4]:
print(len(X_train[7]))

562


In [5]:
word2id = imdb.get_word_index()
id2word = {i: word for word, i in word2id.items()}
print('---review with words---')
print([id2word.get(i, ' ') for i in X_train[6]])
print('---label---')
print(y_train[6])

---review with words---
['the', 'and', 'full', 'involving', 'to', 'impressive', 'boring', 'this', 'as', 'and', 'and', 'br', 'villain', 'and', 'and', 'need', 'has', 'of', 'costumes', 'b', 'message', 'to', 'may', 'of', 'props', 'this', 'and', 'and', 'concept', 'issue', 'and', 'to', "god's", 'he', 'is', 'and', 'unfolds', 'movie', 'women', 'like', "isn't", 'surely', "i'm", 'and', 'to', 'toward', 'in', "here's", 'for', 'from', 'did', 'having', 'because', 'very', 'quality', 'it', 'is', 'and', 'and', 'really', 'book', 'is', 'both', 'too', 'worked', 'carl', 'of', 'and', 'br', 'of', 'reviewer', 'closer', 'figure', 'really', 'there', 'will', 'and', 'things', 'is', 'far', 'this', 'make', 'mistakes', 'and', 'was', "couldn't", 'of', 'few', 'br', 'of', 'you', 'to', "don't", 'female', 'than', 'place', 'she', 'to', 'was', 'between', 'that', 'nothing', 'and', 'movies', 'get', 'are', 'and', 'br', 'yes', 'female', 'just', 'its', 'because', 'many', 'br', 'of', 'overly', 'to', 'descent', 'people', 'time', 

In [6]:
print('Maximum review length: {}'.format(
len(max((X_train + X_test), key=len))))

print('Minimum review length: {}'.format(
len(min((X_test + X_test), key=len))))

Maximum review length: 2697
Minimum review length: 14


In [7]:
from keras.preprocessing import sequence
max_words = 1000
X_train = sequence.pad_sequences(X_train, maxlen=max_words)
X_test = sequence.pad_sequences(X_test, maxlen=max_words)

In [279]:
#DONT RUN THIS
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
num_neurons = 50
embedding_size = 128
model=Sequential()
model.add(Embedding(vocabulary_size, embedding_size, input_length=max_words, name='embedding'))
model.add(LSTM(num_neurons, name='lstm0'))
model.add(Dense(1, activation='sigmoid', name='dense'))
print(model.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 128)         640000    
_________________________________________________________________
lstm0 (LSTM)                 (None, 50)                35800     
_________________________________________________________________
dense (Dense)                (None, 1)                 51        
Total params: 675,851
Trainable params: 675,851
Non-trainable params: 0
_________________________________________________________________
None


In [280]:
model.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy',recall,precision])

In [281]:
batch_size = 64
num_epochs = 1
start = time()

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
model.fit(X_train2, y_train2, validation_data=(X_valid, y_valid), batch_size=batch_size, epochs=num_epochs)

end = time()
print('ELAPSED TIME == ' + str(end - start))

Train on 24936 samples, validate on 64 samples
Epoch 1/1
24936/24936 [==============================] - 397s 16ms/step - loss: 0.4181 - acc: 0.8046 - recall: 0.9974 - precision: 0.4987 - val_loss: 0.2174 - val_acc: 0.9062 - val_recall: 1.0000 - val_precision: 0.5003
ELAPSED TIME == 397.7636468410492


In [282]:
start = time()
scores = model.evaluate(X_test, y_test, verbose=0)
end = time()
print('ELAPSED TIME == ' + str(end - start))
print('Test accuracy:', scores[1])
print('Recall:', scores[2])
print('Precision:', scores[3])

ELAPSED TIME == 118.43455410003662
Test accuracy: 0.85928
Recall: 1.0
Precision: 0.49871665561676026


In [93]:
#2nd model
embedding_size = 128
model2=Sequential()
model2.add(Embedding(vocabulary_size, embedding_size, input_length=max_words, name='embedding'))
model2.add(LSTM(num_neurons, return_sequences=True, name='lstm0'))
model2.add(LSTM(num_neurons, name='lstm1'))
model2.add(Dense(1, activation='sigmoid', name='dense'))
print(model2.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 128)         640000    
_________________________________________________________________
lstm0 (LSTM)                 (None, 1000, 20)          11920     
_________________________________________________________________
lstm1 (LSTM)                 (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 655,221
Trainable params: 655,221
Non-trainable params: 0
_________________________________________________________________
None


In [95]:
model.save_weights('model1_weights.h5')

model2.load_weights('model1_weights.h5', by_name=True) #/Users/hamza/Documents/ceg4136_project/embeddingsize_128/20n

In [96]:
#Check layers are successfully loaded
layer_n = 0
print('MODEL 1')
print(model.layers[layer_n].get_weights())
print('MODEL 2')
print(model2.layers[layer_n].get_weights())

MODEL 1


NameError: name 'model' is not defined

In [97]:
model2.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy',recall,precision])

In [100]:
batch_size = 64
start = time()

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
earlyStop2 = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
model2.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=num_epochs, callbacks=[earlyStop2])

end = time()
print('ELAPSED TIME == ' + str(end - start))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 803s 32ms/step - loss: 0.3400 - acc: 0.8663 - recall: 1.0000 - precision: 0.4997 - val_loss: 0.3238 - val_acc: 0.8670 - val_recall: 1.0000 - val_precision: 0.4985
ELAPSED TIME == 803.0630300045013


In [101]:
start = time()
scores2 = model2.evaluate(X_test, y_test, verbose=0)
end = time()
print('ELAPSED TIME == ' + str(end - start))
print('Test accuracy:', scores2[1])
print('Recall:', scores2[2])
print('Precision:', scores[3])

ELAPSED TIME == 213.9927499294281
Test accuracy: 0.867


NameError: name 'scores' is not defined

In [102]:
#3rd model
embedding_size = 128
model3=Sequential()
model3.add(Embedding(vocabulary_size, embedding_size, input_length=max_words, name='embedding'))
model3.add(LSTM(num_neurons, return_sequences=True, name='lstm0'))
model3.add(LSTM(num_neurons, return_sequences=True, name='lstm1'))
model3.add(LSTM(num_neurons, name='lstm2'))
model3.add(Dense(1, activation='sigmoid', name='dense'))
print(model3.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 128)         640000    
_________________________________________________________________
lstm0 (LSTM)                 (None, 1000, 20)          11920     
_________________________________________________________________
lstm1 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm2 (LSTM)                 (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 658,501
Trainable params: 658,501
Non-trainable params: 0
_________________________________________________________________
None


In [103]:
model2.save_weights('model2_weights.h5')

model3.load_weights('model2_weights.h5', by_name=True)

In [104]:
#Check layers are successfully loaded
layer_n = 0
print('MODEL 2')
print(model2.layers[layer_n].get_weights())
print('MODEL 3')
print(model3.layers[layer_n].get_weights())

MODEL 2
[array([[ 5.4752588e-02,  5.6097824e-02, -1.6529972e-02, ...,
        -3.4603037e-02,  4.3021217e-02,  6.3363209e-02],
       [ 2.4903914e-02,  7.8224838e-03, -3.2543235e-03, ...,
         1.8626127e-02,  3.2737158e-02,  1.5026271e-01],
       [-2.8971089e-02, -2.4517374e-02,  1.9571828e-02, ...,
         1.2287227e-02, -3.5762277e-02,  5.1319592e-02],
       ...,
       [-5.6004334e-02,  4.1572621e-05,  1.3658013e-02, ...,
         3.8546279e-02,  4.4340059e-02, -3.7487511e-02],
       [ 8.6613126e-02,  2.7465805e-02, -1.5148834e-02, ...,
         3.1654868e-02,  4.7615804e-02,  7.7650525e-02],
       [ 1.2962578e-04, -1.6872032e-02,  4.3289773e-02, ...,
         6.2328245e-02, -7.9456195e-03, -1.2363304e-01]], dtype=float32)]
MODEL 3
[array([[ 5.4752588e-02,  5.6097824e-02, -1.6529972e-02, ...,
        -3.4603037e-02,  4.3021217e-02,  6.3363209e-02],
       [ 2.4903914e-02,  7.8224838e-03, -3.2543235e-03, ...,
         1.8626127e-02,  3.2737158e-02,  1.5026271e-01],
       [-

In [105]:
model3.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy',recall,precision])

In [106]:
batch_size = 64
start = time()

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
earlyStop3 = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
model3.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=num_epochs, callbacks=[earlyStop3])

end = time()
print('ELAPSED TIME == ' + str(end - start))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 1225s 49ms/step - loss: 0.2750 - acc: 0.8943 - recall: 0.9974 - precision: 0.4969 - val_loss: 0.3205 - val_acc: 0.8703 - val_recall: 1.0000 - val_precision: 0.4987
ELAPSED TIME == 1226.9113080501556


In [107]:
start = time()
scores3 = model3.evaluate(X_test, y_test, verbose=0)
end = time()
print('ELAPSED TIME == ' + str(end - start))
print('Test accuracy:', scores3[1])
print('Recall:', scores3[2])
print('Precision:', scores3[3])

ELAPSED TIME == 297.5244061946869
Test accuracy: 0.87032


NameError: name 'scores' is not defined

In [108]:
#4rd model
embedding_size = 128
model4=Sequential()
model4.add(Embedding(vocabulary_size, embedding_size, input_length=max_words, name='embedding'))
model4.add(LSTM(num_neurons, return_sequences=True, name='lstm0'))
model4.add(LSTM(num_neurons, return_sequences=True, name='lstm1'))
model4.add(LSTM(num_neurons, return_sequences=True, name='lstm2'))
model4.add(LSTM(num_neurons, name='lstm3'))
model4.add(Dense(1, activation='sigmoid', name='dense'))
print(model4.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 128)         640000    
_________________________________________________________________
lstm0 (LSTM)                 (None, 1000, 20)          11920     
_________________________________________________________________
lstm1 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm2 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm3 (LSTM)                 (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total params: 661,781
Trainable params: 661,781
Non-trainable params: 0
_________________________________________________________________
None

In [109]:
model3.save_weights('model3_weights.h5')

model4.load_weights('model3_weights.h5', by_name=True)

In [110]:
#Check layers are successfully loaded
layer_n = 0
print('MODEL 3')
print(model3.layers[layer_n].get_weights())
print('MODEL 4')
print(model4.layers[layer_n].get_weights())

MODEL 3
[array([[ 0.05086457,  0.06077672, -0.03191191, ..., -0.04059048,
         0.03944267,  0.06057524],
       [ 0.01997495,  0.01285267, -0.03251107, ...,  0.01205484,
         0.0284534 ,  0.17181781],
       [-0.02794131, -0.02438439,  0.04924357, ...,  0.01013155,
        -0.02731604,  0.06314883],
       ...,
       [-0.06081387, -0.00297475,  0.0113614 , ...,  0.04093794,
         0.04074395, -0.04425818],
       [ 0.10360266,  0.02373518, -0.03929824, ...,  0.03273618,
         0.05832445,  0.08889064],
       [ 0.02434014, -0.05831419,  0.05918861, ...,  0.11285921,
        -0.01640111, -0.1450057 ]], dtype=float32)]
MODEL 4
[array([[ 0.05086457,  0.06077672, -0.03191191, ..., -0.04059048,
         0.03944267,  0.06057524],
       [ 0.01997495,  0.01285267, -0.03251107, ...,  0.01205484,
         0.0284534 ,  0.17181781],
       [-0.02794131, -0.02438439,  0.04924357, ...,  0.01013155,
        -0.02731604,  0.06314883],
       ...,
       [-0.06081387, -0.00297475,  0.0113

In [111]:
model4.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy',recall,precision])

In [112]:
start = time()

batch_size = 64
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
earlyStop4 = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
model4.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=num_epochs, callbacks=[earlyStop4])

end = time()
print('ELAPSED TIME == ' + str(end - start))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 1483s 59ms/step - loss: 0.2571 - acc: 0.9006 - recall: 0.9974 - precision: 0.5027 - val_loss: 0.3509 - val_acc: 0.8742 - val_recall: 1.0000 - val_precision: 0.4987
ELAPSED TIME == 1485.9872949123383


In [113]:
start = time()
scores4 = model4.evaluate(X_test, y_test, verbose=0)
end = time()
print('ELAPSED TIME == ' + str(end - start))
print('Test accuracy:', scores4[1])
print('Recall:', scores4[2])
print('Precision:', scores4[3])

ELAPSED TIME == 387.052579164505
Test accuracy: 0.87416
Recall: 1.0
Precision: 0.49925545310974123


In [119]:
#5th model
embedding_size = 128
model5=Sequential()
model5.add(Embedding(vocabulary_size, embedding_size, input_length=max_words, name='embedding'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm0'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm1'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm2'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm3'))
model5.add(LSTM(num_neurons, name='lstm4'))
model5.add(Dense(1, activation='sigmoid', name='dense'))
print(model5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 128)         640000    
_________________________________________________________________
lstm0 (LSTM)                 (None, 1000, 20)          11920     
_________________________________________________________________
lstm1 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm2 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm3 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm4 (LSTM)                 (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total para

In [120]:
model4.save_weights('model4_weights.h5')

model5.load_weights('model4_weights.h5', by_name=True)

In [121]:
#Check layers are successfully loaded
layer_n = 0
print('MODEL 4')
print(model4.layers[layer_n].get_weights())
print('MODEL 5')
print(model5.layers[layer_n].get_weights())

MODEL 4
[array([[ 0.05476777,  0.05980812, -0.05476459, ..., -0.04194805,
         0.04019314,  0.06349309],
       [ 0.02396241,  0.0094558 , -0.06037054, ...,  0.01310516,
         0.02522369,  0.17649397],
       [-0.02566032, -0.02426475,  0.04663971, ...,  0.00978375,
        -0.03006194,  0.06278649],
       ...,
       [-0.07428026, -0.00383674, -0.01454784, ...,  0.0480541 ,
         0.01495041, -0.07813394],
       [ 0.13010933,  0.01749249, -0.04540852, ...,  0.03815641,
         0.06404236,  0.09871268],
       [ 0.02002232, -0.06166923,  0.07075465, ...,  0.12842904,
        -0.02957646, -0.16194957]], dtype=float32)]
MODEL 5
[array([[ 0.05476777,  0.05980812, -0.05476459, ..., -0.04194805,
         0.04019314,  0.06349309],
       [ 0.02396241,  0.0094558 , -0.06037054, ...,  0.01310516,
         0.02522369,  0.17649397],
       [-0.02566032, -0.02426475,  0.04663971, ...,  0.00978375,
        -0.03006194,  0.06278649],
       ...,
       [-0.07428026, -0.00383674, -0.0145

In [122]:
model5.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy',recall,precision])

In [123]:
batch_size = 64
start = time()

X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
earlyStop5 = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
model5.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=num_epochs, callbacks=[earlyStop5])

end = time()
print('ELAPSED TIME == ' + str(end - start))

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 2047s 82ms/step - loss: 0.2390 - acc: 0.9065 - recall: 0.9974 - precision: 0.5023 - val_loss: 0.3381 - val_acc: 0.8726 - val_recall: 1.0000 - val_precision: 0.4987
ELAPSED TIME == 2052.6747467517853


In [124]:
start = time()
scores5 = model5.evaluate(X_test, y_test, verbose=0)
end = time()
print('ELAPSED TIME == ' + str(end - start))
print('Test accuracy:', scores5[1])
print('Recall:', scores5[2])
print('Precision:', scores5[3])

ELAPSED TIME == 533.5545520782471
Test accuracy: 0.87264
Recall: 1.0
Precision: 0.49925545310974123


In [125]:
model5.save_weights('model5_weights.h5')

#model.save('model1.h5')
model2.save('model2.h5')
model3.save('model3.h5')
model4.save('model4.h5')
model5.save('model5.h5')

In [504]:
from keras.callbacks import LearningRateScheduler

def scheduler(epoch):
    if epoch == 5:
        model_galaxy.optimizer.lr.set_value(.02)
    return model_galaxy.optimizer.lr.get_value()

change_lr = LearningRateScheduler(scheduler)

In [ ]:
class EarlyStoppingByLossVal(Callback):
    def __init__(self, monitor='val_loss', value=0.00001, verbose=0):
        super(Callback, self).__init__()
        self.monitor = monitor
        self.value = value
        self.verbose = verbose

    def on_epoch_end(self, epoch, logs={}):
        current = logs.get(self.monitor)
        if current is None:
            warnings.warn("Early stopping requires %s available!" % self.monitor, RuntimeWarning)

        if current < self.value:
            if self.verbose > 0:
                print("Epoch %05d: early stopping THR" % epoch)
            self.model.stop_training = True
callbacks = [
    EarlyStoppingByLossVal(monitor='loss', value=0.00001, verbose=1),
    # EarlyStopping(monitor='val_loss', patience=2, verbose=0),
    ModelCheckpoint(kfold_weights_path, monitor='val_loss', save_best_only=True, verbose=0),
]

In [83]:
import keras
from time import time
import time
class TimeHistory(keras.callbacks.Callback):
    def on_train_begin(self, logs={}):
        import time
        self.times = []

    def on_epoch_begin(self, batch, logs={}):
        self.epoch_time_start = time.time()

    def on_epoch_end(self, batch, logs={}):
        self.times.append(time.time() - self.epoch_time_start)

In [89]:
max_words = 1000
num_neurons = 20
embedding_size = 128
num_epochs = 1

In [90]:
from keras import Sequential
from keras.layers import Embedding, LSTM, Dense, Dropout
#from tensorflow.contrib.rnn import RNNCell
#5 LAYERS TO RUN
model5=Sequential()
model5.add(Embedding(vocabulary_size, embedding_size, input_length=max_words, name='embedding'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm0'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm1'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm2'))
model5.add(LSTM(num_neurons, return_sequences=True, name='lstm3'))
model5.add(LSTM(num_neurons, name='lstm4'))
model5.add(Dense(1, activation='sigmoid', name='dense'))
print(model5.summary())

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 1000, 128)         640000    
_________________________________________________________________
lstm0 (LSTM)                 (None, 1000, 20)          11920     
_________________________________________________________________
lstm1 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm2 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm3 (LSTM)                 (None, 1000, 20)          3280      
_________________________________________________________________
lstm4 (LSTM)                 (None, 20)                3280      
_________________________________________________________________
dense (Dense)                (None, 1)                 21        
Total para

In [91]:
#COMPILE
model5.compile(loss='binary_crossentropy', 
             optimizer='adam', 
             metrics=['accuracy',recall,precision])

batch_size = 64
start = time()

#TRAIN
X_valid, y_valid = X_train[:batch_size], y_train[:batch_size]
X_train2, y_train2 = X_train[batch_size:], y_train[batch_size:]
from keras.callbacks import EarlyStopping
#stop_here_please = EarlyStopping(patience=1)
#time_callback = TimeHistory()
earlyStop = EarlyStopping(monitor='val_loss', min_delta=0, patience=0, verbose=0, mode='auto')
model5.fit(X_train, y_train, validation_data=(X_test, y_test), batch_size=batch_size, epochs=num_epochs, callbacks=[earlyStop])#, callbacks=stop_here_please)

end = time()
print('ELAPSED TIME == ' + str(end - start))

#TEST
start = time()
scores5 = model5.evaluate(X_test, y_test, verbose=0)
end = time()
print('ELAPSED TIME == ' + str(end - start))
print('Test accuracy:', scores5[1])
print('Recall:', scores5[2])
print('Precision:', scores5[3])
# print('Training+Testing time', time_callback.times)

#model5.save_weights('model5_weights_1000w_10n_128es_1epch.h5')
#model.save('model5_weights_1000w_10n_128es_1epch.h5')

Train on 25000 samples, validate on 25000 samples
Epoch 1/1
25000/25000 [==============================] - 1997s 80ms/step - loss: 0.4292 - acc: 0.7997 - recall: 0.9974 - precision: 0.5027 - val_loss: 0.3554 - val_acc: 0.8506 - val_recall: 1.0000 - val_precision: 0.4987
ELAPSED TIME == 2001.1249511241913
ELAPSED TIME == 513.7557771205902
Test accuracy: 0.85064
Recall: 1.0
Precision: 0.49925545310974123
Training+Testing time []


In [27]:
#DATABASE
# import sqlite3
# from sqlite3 import Error, connect
 
# def connectToDb(dbfile)
#     #try:
#         c, conn = sqlite3.connect(dbfile)
#         #print(sqlite3.version)
# #     except sqlite3.Error as e:
# #         print(e)
# #     finally:
# #         conn.close()
#     return c, conn

SyntaxError: invalid syntax (<ipython-input-27-44cfc39abdf7>, line 4)

In [30]:
conn = sqlite3.connect('/Users/hamza/Documents/ceg4136_project/ceg4136.db')
c = conn.cursor()
c.execute('CREATE TABLE growing(id INTEGER PRIMARY KEY, nLayers INTEGER, numNeurons INTEGER, trainingTime REAL, testingTime REAL, max_words INTEGER, embeddingSize INTEGER, withCallbacks text, lossTrain REAL, accTrain REAL, valLoss REAL, valAcc REAL, recallTrain REAL, precisionTrain REAL, valRecall REAL, valPrecision REAL)')


In [53]:
lstmId = 0
c.execute('INSERT INTO growing VALUES(?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?, ?)',(lstmId, 1, num_neurons, 295.0, 110.0, max_words, embedding_size, 'earlyStopping timeHistory', 0.4857, 0.8240, 0.3467, 0.8519, 0.9974, 0.4961, 1.0, 0.4987))


IntegrityError: UNIQUE constraint failed: growing.id

In [62]:
a=c.execute('select * from growing');
print(a.fetchall())

[(0, 1, 20, 295.0, 110.0, 1000, 128, 'earlyStopping timeHistory', 0.4857, 0.824, 0.3467, 0.8519, 0.9974, 0.4961, 1.0, 0.4987)]


In [61]:
c.execute('delete from growing where id=1;')